## Part 1
How many times is the `mul` instruction invoked?

- `set X Y` sets register `X` to the value of `Y`.
- `sub X Y` decreases register `X` by the value of `Y`.
- `mul X Y` sets register `X` to the result of multiplying the value contained in register `X` by the value of `Y`.
- `jnz X Y` jumps with an offset of the value of `Y`, but only if the value of `X` is not zero. (An offset of `2` skips the next instruction, an offset of `-1` jumps to the previous instruction.)

In [1]:
def parse(s):
    try:
        return int(s)
    except ValueError:
        return s

class Processor():
    def __init__(self, instructions):
        self.registers = {chr(ord('a') + i): 0 for i in range(8)}
        self.instructions, self.current_instruction, self.steps, self.mul_count = [], 0, 0, 0
        for ins_s in instructions:
            split = ins_s.split(' ')
            self.instructions.append((split[0], parse(split[1]), parse(split[2])))

    def __str__(self):
        r_print = '\n'.join([f'{register} = {value}' for register, value in self.registers.items()])
        i_print_l = []
        for i in range(len(self.instructions)):
            ins = self.instructions[i]
            instr_p = f'{ins[0]} {ins[1]} {ins[2]}'
            i_print_l.append(f'{i:02}{"*" if i == self.current_instruction else " "}{instr_p}')
        i_print = f'\nInstructions (current = {self.current_instruction:02}):\n' + '\n'.join(i_print_l)
        return f'Current step is {self.steps}, mul was called {self.mul_count} times\n\nRegisters:\n{r_print}\n{i_print}\n'

    def step(self, debug_=False):
        cmd, x, y = self.instructions[self.current_instruction]
        if cmd == 'set':
            self.registers[x] = self.value_of(y)
        elif cmd == 'sub':
            self.registers[x] -= self.value_of(y)
        elif cmd == 'mul':
            self.registers[x] *= self.value_of(y)
            self.mul_count += 1
        elif cmd == 'jnz':
            if self.value_of(x) != 0:
                if debug_:
                    self.debug()
                self.current_instruction += self.value_of(y)
                self.steps += 1
                return
        else:
            raise ValueException(f'No such command: {cmd}')
        if debug_:
            self.debug()
        self.current_instruction += 1
        self.steps += 1

    def value_of(self, y):
        return y if isinstance(y, int) else self.registers[y]

    def run(self, max_steps=10000, debug=False):
        for i in range(max_steps):
            if self.current_instruction > len(self.instructions) - 1:
                return
            self.step(debug)
        raise Exception(f'Execution have not ended after {self.steps} steps. Processor dump:\n{self}')

    def debug(self):
        i = self.instructions[self.current_instruction]
        instr_p = f'{i[0]} {i[1]} {i[2]}'.ljust(14)
        regs = ''.join([f'{reg}={val}'.ljust(13) for reg, val in self.registers.items()])
        print(f'{self.steps + 1:03} {self.current_instruction:02} {instr_p} {regs}')

In [2]:
puzzle_in = [line[:-1] for line in open('in/day23.txt', 'r')]
proc = Processor(puzzle_in)
proc.run(100000)
print(f'Part 1 answer is: `mul` was called {proc.mul_count} times')
assert proc.mul_count == 6724

Part 1 answer is: `mul` was called 6724 times


## Part 2
After setting register `a` to `1`, if the program were to run to completion, what value would be left in register `h`?

### Simply replacing some instructions to avoid loops didn't help
Because it was missing the point - count the non-prime numbers between `108400` and `125400` with step `17`.

In [3]:
ins = ['set a 1'] + puzzle_in
ins[20] = 'set e b'
ins[24] = 'set d b'
proc = Processor(ins)
proc.run(2000000, False)
print(proc)

Current step is 21028, mul was called 1002 times

Registers:
a = 1
b = 125400
c = 125400
d = 125400
e = 125400
f = 1
g = 0
h = 0

Instructions (current = 33):
00 set a 1
01 set b 84
02 set c b
03 jnz a 2
04 jnz 1 5
05 mul b 100
06 sub b -100000
07 set c b
08 sub c -17000
09 set f 1
10 set d 2
11 set e 2
12 set g d
13 mul g e
14 sub g b
15 jnz g 2
16 set f 0
17 sub e -1
18 set g e
19 sub g b
20 set e b
21 sub d -1
22 set g d
23 sub g b
24 set d b
25 jnz f 2
26 sub h -1
27 set g b
28 sub g c
29 jnz g 2
30 jnz 1 3
31 sub b -17
32 jnz 1 -23



### Attempted answers to part 2
`902` is too low - because `range()` does not include the second argument.
`903` is the right answer.

In [4]:
h = 0
for b in range(108400, 125400+1, 17):
    for c in range(2, b):
        if b % c == 0:
            h += 1
            break
print(f'Answer to part 2 is: {h}')  # 903

Answer to part 2 is: 903
